#  Хэш-функции и их применение.

## Цели работы
1. Определение Хэш-функции.
2. Проверка целостности файлов.
3. Хранение паролей.
4. Поиск подстроки в строке алгоритмом Рабина—Карпа.

## Хэш-функция
Хэш-функция — эта функция, которая преобразует строку (последовательность символов) в число. Основным требованием к хэш-функции является требование равномерного распределения значений. Отсюда и название — `хэширование` (`hashing` от англ. «перемешивание»).

Вторым требованием к Хэш-функции является уникальность значений функции для всех значений аргумента. На практике возможно соответствие значения функции нескольким значениям аргумента — так называемая **коллизия**.

Важным свойством с точки зрения приложений кибербезопасности Хэш-функции является невозможность восстановления значения аргумента по известному значению функции за разумное время.
### Библиотека hashlib
Для вычисления хэш-функции в python используются функции библиотеки [hashlib](https://docs.python.org/3/library/hashlib.html). Например, код ниже вычисляет хэш-функцию строки `Hello world!` с помощью алгоритма *SHA*. Запустите код в ячейке ниже:

In [ ]:
import hashlib
m = hashlib.sha256()
m.update(b"Hello world!")
print(m.hexdigest())

## Применение Хэш-функции
### Проверка целостности файлов
Одно из распространенных применений Хэш-функции — это подтверждение целостности файлов при передаче их по каналам связи/скачивания с веб-ресурсов. Если значение Хэш-функции полученного по сети файла совпадает с эталонным значением, значит файл передан без искажений.
Код в ячейке ниже осуществляет чтение файла и вычисление MD5 значения Хэш-функции:
```python
import hashlib
md5_hash = hashlib.md5()
with open("file.txt", "rb") as f:
    while True:
        data = f.read(2048)
        if not data:
            break
        md5_hash.update(data)
print(md5_hash.hexdigest())
```
**Задание 1 —** создайте текстовый файл и вычислите его md5 хэш с помощью примера кода выше.

In [2]:
# Код для создания файла file.txt (если он еще не существует)
try:
    with open("file.txt", "w", encoding="utf-8") as f:
        f.write("Это тестовый файл для вычисления хэша.\n")
        f.write("Добавьте сюда еще несколько строк текстаукеуеуе.")
    print("Файл 'file.txt' успешно создан.")
except IOError as e:
    print(f"Ошибка при создании файла: {e}")

# Теперь выполним код для вычисления MD5 хэша файла
import hashlib

md5_hash = hashlib.md5()
try:
    # Чтение файла в бинарном режиме 'rb'
    with open("file.txt", "rb") as f:
        # Читаем файл по частям, чтобы обработать большие файлы
        while True:
            data = f.read(2048) # Читаем 2048 байт за раз
            if not data:
                break # Если данных больше нет, выходим из цикла
            md5_hash.update(data) # Обновляем хэш данными

    print("\nMD5 хэш файла 'file.txt':")
    print(md5_hash.hexdigest())

except FileNotFoundError:
    print("Ошибка: файл 'file.txt' не найден. Пожалуйста, убедитесь, что он существует.")
except IOError as e:
    print(f"Ошибка при чтении файла: {e}")

Файл 'file.txt' успешно создан.

MD5 хэш файла 'file.txt':
84adfc9287fefdb6c1e9972a51c3b6f7


### Хранение паролей
Другим распространенным применением Хэш-функции является безопасное хранение паролей. В этом случае хранят не сами пароли, которые пользователь ввел, например, при регистрации на сайте, а результат вычисления их хэш-функции.

Затем на этапе аутентификации пользователя сравниваются хэш значения введенного пароля и хэш значение, сохраненное в базе.

Для генерации хэша пароля можно использовать функцию `pbkdf2_hmac()` из библиотеки `hashlib`. Функция `pbkdf2_hmac()` принимает пять параметров:
 - hash_name: алгоритм хеш дайджеста для HMAC;
 - password: пароль, превращенный в ключ;
 - salt: случайно сгенерированная соль;
 - iterations: итерации в вычислении (чем больше, тем длиннее вычисления);
 - dklen: длина ключа вывода (не обязательно).

Перед генерацией ключа с использованием `pbkdf2_hmac` нужно сгенерировать случайную соль. Смешивание исходного пароля с солью увеличивает число возможных вариантов перебора и затрудняет подбор пароля по известному хэш методом грубой силы. Использование соли требует чуть больше работы и хранении дополнительной последовательности байтов. Для генерации соли используется функция `os.urandom()`, которая возвращает случайные байты, используемые для шифрования. Соль складывается с паролем, чтобы ввод покрывал больший диапазон.

Пример кода генерации соли:
```python
import os
salt = os.urandom(32)
```
Параметр 32 является размером, возвращаемым в байтах.

Ниже пример кода генерации хэш-функции от пароля:
```python
import hashlib
import os

salt = os.urandom(32)
password = 'QWasZX12'
hash_pass = hashlib.pbkdf2_hmac('sha256', password.encode('utf-8'), salt, 100000)
```
Использованы следующие параметры функции `hashlib.pbkdf2_hmac`:
 - 'sha256' — алгоритм хеширования
 - password.encode('utf-8') — пароль в виде байтов
 - salt — Соль
 - число итераций алгоритма хэширования, затрудняющее подбор пароля

Для последующего сравнения введенного пользователем пароля необходимо хранить в базе данных хэш и соль.

**Задание 2 —** Реализуйте пользовательский ввод пароля и сравнение с сохраненным эталоном.

In [3]:
import hashlib
import os

# --- Часть 1: Имитация сохранения пароля при регистрации ---
# Этот код выполняется один раз, когда пользователь устанавливает свой пароль

# Генерируем уникальную соль для этого пользователя
saved_salt = os.urandom(32)

# Имитируем пароль, который пользователь ввел при регистрации
original_password = 'QWasZX12'

# Хэшируем пароль с использованием соли и большого числа итераций
# pbkdf2_hmac('алгоритм', пароль (байты), соль (байты), число итераций)
saved_hash = hashlib.pbkdf2_hmac(
    'sha256',              # Алгоритм хеширования
    original_password.encode('utf-8'), # Пароль в виде байтов
    saved_salt,            # Сгенерированная соль
    100000                 # Число итераций (большое число для замедления брутфорса)
)

# В реальной системе saved_salt и saved_hash были бы сохранены в базе данных
print("--- Имитация сохраненных учетных данных ---")
# Выведем соль и хэш в шестнадцатеричном виде для наглядности (в БД хранятся байты)
print(f"Сохраненная соль (bytes): {saved_salt.hex()}...")
print(f"Сохраненный хэш (bytes): {saved_hash.hex()}...")
print("-" * 40)

# --- Часть 2: Имитация проверки пароля при входе ---
# Этот код выполняется каждый раз, когда пользователь пытается войти

# Получаем пароль от пользователя
user_input_password = input("Введите пароль для проверки: ")

# Теперь, чтобы проверить введенный пароль, мы должны:
# 1. Получить сохраненную соль для этого пользователя (у нас она в переменной saved_salt)
# 2. Хэшировать ВВЕДЕННЫЙ пользователем пароль, используя ТУ ЖЕ сохраненную соль и параметры

input_hash = hashlib.pbkdf2_hmac(
    'sha256',              # Тот же алгоритм
    user_input_password.encode('utf-8'), # Введенный пароль в виде байтов
    saved_salt,            # ИСПОЛЬЗУЕМ СОХРАНЕННУЮ СОЛЬ
    100000                 # То же число итераций
)

# 3. Сравнить вычисленный хэш введенного пароля с сохраненным хэшем
if input_hash == saved_hash:
    print("Пароль верный!")
else:
    print("Пароль неверный.")

--- Имитация сохраненных учетных данных ---
Сохраненная соль (bytes): d6597825c4f1cab275740b6f88870fd5fdf0d92c55392de48f7c18ca13ff5486...
Сохраненный хэш (bytes): 31b2b2ff6dce749944f2bd6971c07e859d980e84abad5a6dcce1413f7d1c7322...
----------------------------------------
Введите пароль для проверки: QWasZX12
Пароль верный!


### Поиск подстроки в строке алгоритмом Рабина—Карпа
В осеннем семестре рассматривался "наивный" алгоритм поиска подстроки в строке с квадратичным временем исполнения в наихудшем случае $O(n^{2})$, который заключался в последовательном наложении подстроки `p` на строку `s`.

На этом занятии будет рассмотрен алгоритм поиска подстроки `p` в строке `s` с линейным временем выполнения $O(n)$ — алгоритм Рабина—Карпа, основанный на применении Хэш-функции.

Алгоритм основан на том факте, что если строка-образец `p` (длины *m*) и подстрока длины *m* символов, начинающаяся с позиции *i* строки `s` совпадают, то должны совпадать и их хэш-функции. Если же строки разные, то и их хэш-функции почти наверняка отличаются. Одинаковые хэш-функции для разных строк должны встречаться настолько редко, что можно допустить время $O(m)$ на явную проверку идентичности строк при совпадении хэш-функций. Это сводит сложность поиска подстроки в строке к вычислениям $n-m+2$ значений хэш-функции плюс небольшое число сравнений сложности $O(m)$ строк с одинаковыми значениями хэш-функций.

Надлежащий выбор хэш-функции позволит тратить на ее вычисление для разных отрезков строки `s` время, меньшее $O(m)$. Таким правильным выбором является хеш-функция `m` символов строки `s`, начинающихся с символа `j`:
$$H(s, j) = (\sum_{i=0}^{m-1} \alpha^{m-(i+1)}\times ord(s_{i+j})) mod{q},$$ где $ord(c)$ — ANSI-код символа `c`, q — простое число, а $\alpha$ — число от 0 до q-1.

Удобство выбора именно такой хэш-функции заключается в том, что $H(s, j+1)$ (т.е. хэш-функция от подстроки `s` длины `m`, сдвинутой на 1 символ) равна: $$H(s, j+1) = (\alpha(H(s, j)-\alpha^{m-1}ord(s_j)) + ord(s_{j+m})) mod {q}.$$
Другими словами, если нам уже известна хэш-функция $H(s, j)$ для подстроки, начинающейся в позиции `j`, то хэш-функцию подстроки с позиции `j+1` можно вычислить выполнив две операции умножения, одну операцию сложения, одну операцию вычитания и одну операцию вычисления остатка от деления.  
Одним из возможных эффективных методов выбора пары $\alpha$ и q [следующий](https://ru.wikipedia.org/wiki/Алгоритм_Рабина_—_Карпа): $\alpha = 2,$ q — случайное число из диапазона $[2...n^3]$.

Возможная реализация алгоритма Рабина-Карпа на *Python* выглядит следующим образом (принимает на вход строку `s` и образец `p`, возвращает индекс начала искомого образца в строке, либо -1, если образец не найден):
```python
def rabin_karp(s, p):
    n = len(s)
    m = len(p)
    hash_p = hash_func(p);
    for i in range(n-m+1):
        hash_s = hash_func(s[i:i+m])
        if hash_s == hash_p:
            if s[i:i+m] == p:
                return i
    return -1
```
**Задание 3 —** Реализуйте функцию `hash_func()` в соответствии с описанием выше и скорректируйте код функции `rabin_karp(s, p)` из примера выше, чтобы получить работающую реализацию алгоритма Рабина-Карпа. Продемонстрируйте ее работу.

In [ ]:
# --- Реализация хэш-функции для алгоритма Рабина-Карпа ---

def hash_func(s, prime_base=31, modulus=10**9 + 7):
    """
    Простая полиномиальная хэш-функция для строки.
    s: входная строка
    prime_base: простое основание для хэширования
    modulus: большой простой модуль
    """
    h = 0
    # Вычисление хэша: (c1*p^(m-1) + c2*p^(m-2) + ... + cm*p^0) mod M
    # Или итеративно: h = (h * prime_base + ord(char)) % modulus
    for char in s:
        h = (h * prime_base + ord(char)) % modulus
    return h

# --- Реализация алгоритма Рабина-Карпа ---

def rabin_karp(s, p, prime_base=31, modulus=10**9 + 7):
    """
    Алгоритм поиска подстроки Рабина-Карпа с простой хэш-функцией.
    Примечание: Эта реализация следует структуре из задания и ПЕРЕСЧИТЫВАЕТ
    хэш каждого окна, а не использует эффективный перекатывающийся хэш.
    При совпадении хэшей выполняет посимвольную проверку для избежания коллизий.

    s: текст (строка)
    p: шаблон (строка)
    prime_base: простое основание для хэширования
    modulus: большой простой модуль
    """
    n = len(s)
    m = len(p)
    if m > n:
        return -1 # Шаблон длиннее текста, совпадение невозможно

    # Вычисляем хэш шаблона один раз
    hash_p = hash_func(p, prime_base, modulus)

    # Итерируемся по всем возможным начальным позициям шаблона в тексте
    for i in range(n - m + 1):
        # Вычисляем хэш текущего "окна" текста размером с шаблон
        hash_s = hash_func(s[i:i+m], prime_base, modulus)

        # Если хэши совпадают, выполняем посимвольную проверку
        # Это необходимо для обработки коллизий хэш-функции (когда разные строки имеют одинаковый хэш)
        if hash_s == hash_p:
            # Проверяем, действительно ли подстрока совпадает с шаблоном
            if s[i:i+m] == p:
                return i # Шаблон найден по индексу i

    # Если цикл завершился и совпадений не найдено
    return -1

# --- Демонстрация работы ---

text = "Это демонстрационная строка для алгоритма поиска подстроки Рабина-Карпа."
pattern1 = "алгоритма" # Шаблон, который присутствует в тексте
pattern2 = "быстрый"   # Шаблон, который отсутствует в тексте

print("\n--- Демонстрация алгоритма Рабина-Карпа ---")

# Ищем первый шаблон
index1 = rabin_karp(text, pattern1)
if index1 != -1:
    print(f"Шаблон '{pattern1}' найден по индексу: {index1}")
    print(f"Найденная подстрока: '{text[index1 : index1 + len(pattern1)]}'")
else:
    print(f"Шаблон '{pattern1}' не найден.")

print("-" * 30)

# Ищем второй шаблон
index2 = rabin_karp(text, pattern2)
if index2 != -1:
    print(f"Шаблон '{pattern2}' найден по индексу: {index2}")
else:
    print(f"Шаблон '{pattern2}' не найден.")


--- Демонстрация алгоритма Рабина-Карпа ---
Шаблон 'алгоритма' найден по индексу: 32
Найденная подстрока: 'алгоритма'
------------------------------
Шаблон 'быстрый' не найден.


В этом коде мы сначала определили простую hash_func. Затем в функции rabin_karp мы использовали эту хэш-функцию для вычисления хэша шаблона и хэша каждого скользящего окна в тексте. При совпадении хэшей выполняется прямое сравнение подстроки и шаблона (s[i:i+m] == p), чтобы исключить ложные срабатывания из-за коллизий хэша. Демонстрационная часть показывает поиск существующей и несуществующей подстроки.

# Индивидуальные задания

| Вариант | Задание 1 (Основы и hashlib) | Задание 2 (Применение 1: Целостность файлов / Хранение паролей) | Задание 3 (Применение 2: Рабин-Карп / Комбинированное) |
|---|---|---|---|
| 1 | Что такое хэш-функция согласно тексту? Перечислите ее основные требования. | Создайте файл `test1.txt` с любым текстом. Вычислите и выведите его MD5 хэш, используя пример из текста. | Объясните, почему при поиске подстроки алгоритмом Рабина-Карпа при совпадении хэшей все равно нужна проверка самих строк. |
| 2 | Какое свойство хэш-функции особенно важно для кибербезопасности? Почему? | Напишите код, который запрашивает у пользователя пароль, генерирует для него соль (16 байт) и вычисляет хэш с помощью `pbkdf2_hmac` и `sha256` (50000 итераций). Выведите соль и хэш. | В чем заключается "удобство" выбора формулы хэш-функции для алгоритма Рабина-Карпа при переходе к следующему сегменту строки? |
| 3 | Что такое коллизия хэш-функции? Является ли это проблемой на практике? | Создайте файл `data.bin` (можно создать его программно, записав в него несколько байт, например `b"sample data"`). Вычислите и выведите его SHA1 хэш (подсказка: используйте `hashlib.sha1()`). | Реализуйте часть алгоритма Рабина-Карпа: напишите функцию `simple_hash(text_segment)` которая вычисляет сумму ASCII-кодов символов в `text_segment`. Продемонстрируйте ее работу на примере. |
| 4 | Используя библиотеку `hashlib`, вычислите SHA256 хэш для строки "Hello Python". Выведите результат. | Объясните, зачем нужна "соль" (salt) при хэшировании паролей. | В функции `rabin_karp(s, p)` из примера, что произойдет, если `hash_func` всегда будет возвращать 0? Как это повлияет на производительность? |
| 5 | Какую функцию из `hashlib` можно использовать для генерации хэша пароля, учитывая соль и итерации? Перечислите ее основные параметры. | Создайте файл `secrets.txt` с одной строкой "This is a secret". Вычислите его MD5 хэш. Затем измените один символ в файле и снова вычислите MD5 хэш. Сравните результаты. | В алгоритме Рабина-Карпа, зачем используется `mod q` при вычислении хэша? |
| 6 | Объясните своими словами, что делает метод `update()` у объекта хэш-функции в `hashlib`. | Напишите код для "регистрации" пользователя: запросите пароль, сгенерируйте соль, вычислите хэш (`pbkdf2_hmac`, `sha256`, 100000 итераций). Сохраните (просто выведите на экран) соль и хэш. | Даны строка `s = "abracadabra"` и образец `p = "abr"`. Вычислите "наивный" хэш (сумма ASCII-кодов) для `p` и для первых трех символов `s`. |
| 7 | Почему при работе с файлами в примере кода для вычисления хэша файл читается по частям (`f.read(2048)`)? | Напишите код для "аутентификации" пользователя: предположим, у вас есть сохраненные соль `stored_salt` и хэш `stored_hash` (задайте их константами, например, из предыдущего задания). Запросите пароль у пользователя, вычислите его хэш с использованием `stored_salt` и сравните с `stored_hash`. | Какова сложность "наивного" алгоритма поиска подстроки в строке в наихудшем случае? Какую сложность стремится достичь алгоритм Рабина-Карпа? |
| 8 | Используя `hashlib`, вычислите MD5 хэш для строки "Cryptography". Выведите результат. | Что такое `os.urandom(32)` и для чего он используется в контексте хэширования паролей? | Если бы вы реализовывали `hash_func()` для Рабина-Карпа, какие параметры, кроме самой строки, ей бы понадобились согласно описанию? (α, q) |
| 9 | Каково основное требование к хэш-функции, связанное с распределением ее значений? | Измените пример кода для вычисления хэша файла так, чтобы он использовал алгоритм SHA512 (`hashlib.sha512()`). Протестируйте на любом созданном вами файле. | Предположим, `hash_func` для Рабина-Карпа уже реализована. Напишите псевдокод или объясните словами, как бы вы модифицировали функцию `rabin_karp(s, p)` для подсчета *всех* вхождений `p` в `s`. |
| 10 | Что означает "невозможность восстановления значения аргумента по известному значению функции за разумное время"? | Почему важно хранить не только хэш пароля, но и соль, использованную при его генерации? | В формуле для `H(s,j+1)` в алгоритме Рабина-Карпа, какой член "удаляет" вклад первого символа предыдущего окна, и какой "добавляет" вклад нового символа? |
| 11 | Вычислите SHA1 хэш строки "Hashing is fun!" с помощью `hashlib`. | Запросите у пользователя две строки. Вычислите для каждой MD5 хэш. Сравните хэши и сообщите, совпадают ли строки (без прямого сравнения самих строк, только по хэшам). | Что будет, если в алгоритме Рабина-Карпа параметр `q` (простое число) выбрать слишком маленьким? |
| 12 | Приведите пример использования `m.update(b"Hello")` и `m.update(b" world!")`. Какой будет итоговый хэш по сравнению с `m.update(b"Hello world!")`? Проверьте кодом для SHA256. | Напишите функцию `generate_password_hash(password_str)` которая принимает пароль, генерирует соль (32 байта), использует `pbkdf2_hmac` с `sha256` и 120000 итераций, и возвращает кортеж `(salt, hashed_password)`. | В описании `hash_func` для Рабина-Карпа упоминается `ord(c)`. Что это за функция и что она возвращает? |
| 13 | Перечислите три основных применения хэш-функций, упомянутых в тексте. | Реализуйте простой "менеджер паролей" на уровне концепции: функция "сохранить пароль" (генерирует соль и хэш, выводит их) и функция "проверить пароль" (принимает пароль, соль, эталонный хэш, возвращает True/False). | Почему в алгоритме Рабина-Карпа используется число `α` (альфа)? Какую роль оно играет в формуле хэша? |
| 14 | Какой алгоритм хэширования используется в примере кода для проверки целостности файлов? | Сгенерируйте две разные соли с помощью `os.urandom(16)`. Выведите их. Будут ли они одинаковыми? Почему? | Если `hash_func(s[i:i+m]) == hash_p` в алгоритме Рабина-Карпа, гарантирует ли это, что `s[i:i+m] == p`? Почему или почему нет? |
| 15 | Как называется ситуация, когда двум разным входным строкам соответствует одно и то же значение хэш-функции? | Напишите код, который создает два файла `fileA.txt` и `fileB.txt` с одинаковым содержимым. Вычислите и сравните их MD5 хэши. | Зачем в алгоритме Рабина-Карпа рекомендуется выбирать `q` как простое число? |
| 16 | Запустите пример кода `m = hashlib.sha256(); m.update(b"Hello world!"); print(m.hexdigest())`. Какой результат вы получили? | Объясните, что означает параметр `iterations` в функции `pbkdf2_hmac()` и как его изменение влияет на безопасность. | Модифицируйте предоставленный прототип `rabin_karp(s,p)` так, чтобы он использовал "наивный" хэш (сумму ASCII кодов) вместо `hash_func`. Протестируйте. |
| 17 | В чем отличие функции `pbkdf2_hmac()` от простого вычисления, например, `sha256()` от пароля? | Если злоумышленник получит доступ к базе данных, где хранятся хэши паролей и соответствующие им соли, сможет ли он легко восстановить исходные пароли? Почему? | Для строки `s = "test"` и `m=2`, `α=2`, `q=101` (для простоты), вычислите хэш `H(s,0)` по формуле Рабина-Карпа: `(α*ord(s[0]) + ord(s[1])) mod q`. |
| 18 | Какую кодировку (`encode`) нужно применить к строке пароля перед передачей в `pbkdf2_hmac`? Почему это необходимо? | Создайте файл `image.jpg` (можно просто переименовать любой текстовый файл для этого задания). Вычислите его SHA256 хэш. | В алгоритме Рабина-Карпа, если хэш образца `p` равен 12345, и хэш текущего окна в строке `s` тоже 12345, что должен сделать алгоритм дальше? |
| 19 | Назовите хотя бы два разных алгоритма хэширования, доступных в библиотеке `hashlib`. | Пользователь забыл пароль. У вас есть только хэш его пароля и соль. Можете ли вы восстановить пароль для него? Объясните. | Реализуйте функцию `hash_func(text_segment, alpha, q)` согласно первой формуле для Рабина-Карпа (не рекуррентной). `H(s,j)=(∑i=0m−1αm−(i+1)×ord(si+j))modq`. Протестируйте для `text_segment="ab"`, `alpha=2`, `q=101`. |
| 20 | Объясните принцип проверки целостности файлов с помощью хэш-функций. | Напишите код, который генерирует и выводит 3 разные соли размером 8 байт каждая. | Что означает `dklen` в параметрах `pbkdf2_hmac`? Является ли он обязательным? |
| 21 | Вычислите SHA256 хэш для пустой строки `""`. | Предположим, вы скачали программу и ее заявленный MD5 хэш. Как вы проверите, что программа не была повреждена при скачивании? Опишите шаги. | Опишите, как вычисляется `H(s,j+1)` зная `H(s,j)` в алгоритме Рабина-Карпа. Какие операции для этого нужны? |
| 22 | Что такое "равномерное распределение значений" для хэш-функции и почему это важно? | Если бы вы не использовали соль при хэшировании паролей, как это могло бы упростить задачу злоумышленнику, имеющему доступ к таблице хэшей (например, с помощью "радужных таблиц")? | Модифицируйте код `rabin_karp(s, p)` из текста так, чтобы вместо `hash_func(s[i:i+m])` он использовал "скользящий хэш" согласно формуле для `H(s,j+1)`, но для простоты можете считать, что `hash_func` на первой итерации вычисляет хэш "полностью", а на последующих использует рекуррентную формулу (реализовывать саму `hash_func` не обязательно, просто показать структуру цикла). |
| 23 | Для чего используется параметр `hash_name` в функции `pbkdf2_hmac()`? Приведите пример значения. | Создайте два файла: `original.txt` с текстом "Hello" и `corrupted.txt` с текстом "Hallo". Вычислите для обоих SHA1 хэши и сравните их. | В алгоритме Рабина-Карпа, какой выбор `α` и `q` предлагается как "один из возможных эффективных методов"? |
| 24 | Можно ли по хэш-значению, полученному, например, с помощью SHA256, однозначно восстановить исходную строку? Почему? | Зачем увеличивать число итераций (`iterations`) в `pbkdf2_hmac`? Каково последствие (кроме увеличения безопасности)? | Представьте, что `hash_func` для Рабина-Карпа подвержена частым коллизиям. Как это скажется на производительности алгоритма `rabin_karp`? |
| 25 | Используя `hashlib`, вычислите SHA256 хэш для строки "Final Test!" и затем, используя тот же объект хэша, добавьте строку " And More!" с помощью `update()`. Выведите итоговый хэш. | Напишите скрипт, который принимает имя файла как ввод от пользователя, вычисляет его MD5 хэш и выводит результат. | Если в алгоритме Рабина-Карпа `m` (длина образца) равна `n` (длина строки), сколько раз будет вычислена хэш-функция для подстроки `s`? |

In [4]:
#Задание 2 
import hashlib
def create_file(filename, content):
    """
    Создаёт файл с указанным именем и записывает в него заданное содержимое.
    """
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(content)

def compute_md5(filename):
    """
    Вычисляет MD5-хэш файла.
    """
    hasher = hashlib.md5()
    with open(filename, 'rb') as f:
        # Читаем файл блоками по 4096 байт
        for chunk in iter(lambda: f.read(4096), b''):
            hasher.update(chunk)
    return hasher.hexdigest()

def main():
    content_a = "Пример одинакового содержимого.\n"
    file_a = 'fileA.txt'
    file_b = 'fileB.txt'

    # Создаём два файла с одинаковым содержимым
    create_file(file_a, content_a)
    create_file(file_b, content_b)

    # Вычисляем MD5-хэши обоих файлов
    hash_a = compute_md5(file_a)
    hash_b = compute_md5(file_b)

    # Выводим хэши
    print(f"MD5 {file_a}: {hash_a}")
    print(f"MD5 {file_b}: {hash_b}")

    # Сравниваем хэши
    if hash_a == hash_b:
        print("Хэши совпадают файлы идентичны.")
    else:
        print("Хэши различаются содержимое файлов разное.")

if __name__ == "__main__":
    main()




NameError: name 'content_b' is not defined